# Pinhole camera calibration for OpenVSLAM

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import sys
import os
from os.path import join
import yaml
import glob
from functools import partial

from enum import Enum
class Camera(Enum):
    OpenCV = 1 # OpenCV
    PySpin = 2 # Spinnaker
    
class Pattern(Enum):
    Chessboard = 1
    Circle = 2

%matplotlib inline

# Capture checkboard images
Checkborad pdf file is available at 
 - https://raw.githubusercontent.com/opencv/opencv/master/doc/pattern.png
 - https://raw.githubusercontent.com/opencv/opencv/master/doc/acircles_pattern.png

In [ ]:
# User setting

# Input camera
cam_kind = Camera.OpenCV
# cam_kind = Camera.PySpin

# Pattern
pattern_kind = Pattern.Chessboard
# pattern_kind = Pattern.Circle


In [ ]:
# Find pattern
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
if pattern_kind == Pattern.Chessboard:
    pattern_size = (6,9)
    detect_flag = cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_NORMALIZE_IMAGE # cv2.CALIB_CB_FAST_CHECK+
    findCorners = partial(cv2.findChessboardCorners, patternSize=pattern_size, flags=detect_flag)
elif pattern_kind == Pattern.Circle:
    pattern_size = (4, 11)
    detect_flag = cv2.CALIB_CB_ASYMMETRIC_GRID+cv2.CALIB_CB_CLUSTERING+cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_NORMALIZE_IMAGE
    findCorners = partial(cv2.findCirclesGrid, patternSize=pattern_size, flags=detect_flag)

# Save images if corners are detected
checkBeforeInsert = True


In [ ]:
cam_id = 0

if cam_kind == Camera.PySpin:
    sys.path.insert(0, './SpinnakerVideoCapture/python')
    from PySpinCap import PySpinManager
    # start manager
    manager = PySpinManager()
    cap = manager.get_camera(cam_id)
if cam_kind == Camera.OpenCV:
    cap = cv2.VideoCapture(cam_id)
    fps = cap.get(cv2.CAP_PROP_FPS)
    print('Framerate:',fps)


In [ ]:
print('#####################')
print('### start capturing')
elps = []
save_imgs = []
# cv2.namedWindow('show img', cv2.WINDOW_KEEPRATIO)
while True:
    start = time.time()

    ret, img = cap.read()
    cv2.imshow('show img', img)
#     show = cv2.resize(img, None, fx=0.5, fy=0.5)
#     cv2.imshow('show img', show)
    key = cv2.waitKey(20)
    elps.append((time.time() - start))

    if key == ord('s'):
        cv2.waitKey(300)
        
        ret = True
        if checkBeforeInsert:
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            # Find the chess board corners
            ret, corners = findCorners(gray)
        if ret:
            save_imgs.append(img)
            print('save image:', len(save_imgs))
        else:
            print("couldn't find pattern")
    elif key == 27:
        break

    if len(elps) == 100:
        print('- FPS:{}'.format(len(elps) / sum(elps)))
        elps = []

cv2.destroyAllWindows()
print('### finish capturing')
print('#####################')


In [ ]:
# Release everything
cap.release()
if cam_kind == Camera.PySpin:
    manager.release()

In [ ]:
save_img_folder = './img{}'.format(time.strftime("%Y%m%d-%H%M"))
try:
    os.mkdir(save_img_folder)
except OSError as exc:
    print(exc)
print(save_img_folder)

In [ ]:
# Save images
for i,it in enumerate(save_imgs):
    print('save image : {:03}'.format(i))
    cv2.imwrite(join(save_img_folder, '{:03}.png'.format(i)), it)

# Calib image

## Select pattern type first

In [ ]:
# load_folder = './img20191027-2136/'
# load_folder = './img20191027-2312/'
load_folder = save_img_folder
print('load folder:', load_folder)

In [ ]:
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
%matplotlib inline

images = []
for fname in sorted(glob.glob('{}/*.png'.format(load_folder))):
    img = cv2.imread(fname)
    images.append(img)
    print(fname)

In [ ]:
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
imgsize = None
if pattern_kind == Pattern.Chessboard:
    pattern_size = (6,9)
    objp = np.zeros((1, pattern_size[0]*pattern_size[1], 3), np.float32)
    objp[0,:,:2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
    detect_flag = cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_NORMALIZE_IMAGE # cv2.CALIB_CB_FAST_CHECK+
    findCorners = partial(cv2.findChessboardCorners, patternSize=pattern_size, flags=detect_flag)
    
elif pattern_kind == Pattern.Circle:
    pattern_size = (4, 11)
    objp = []
    for i in range(pattern_size[1]):
        for j in range(pattern_size[0]):
            objp.append([2*j+i%2, i ,0])
    objp = np.array(objp, np.float32)[np.newaxis]
    detect_flag = cv2.CALIB_CB_ASYMMETRIC_GRID+cv2.CALIB_CB_CLUSTERING
    findCorners = partial(cv2.findCirclesGrid, patternSize=pattern_size, flags=detect_flag)


In [ ]:
# check first image
if True:
    img = images[2].copy()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = findCorners(gray)

    if ret:
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        cv2.drawChessboardCorners(img, pattern_size, corners, ret) 
        x_max, y_max = np.max(corners, axis=0)[0]
        x_min, y_min = np.min(corners, axis=0)[0]
        offset = 50
        x_max = int(x_max)+offset
        x_min = int(x_min)-offset
        y_max = int(y_max)+offset
        y_min = int(y_min)-offset
        fig, ax = plt.subplots(1, 2, figsize=(15,8))
        ax[0].imshow(img[y_min:y_max,x_min:x_max,::-1])
        ax[1].imshow(img[:,:,::-1])
    else:
        print("couldn't find corner")

In [ ]:
save_cornerimg = True

objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
for i, img in enumerate(images):
    if imgsize == None:
        imgsize = img.shape[:2]
    else:
        assert imgsize == img.shape[:2], "All images must share the same size."
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = findCorners(gray)
    # If found, add object points, image points (after refining them)
    if ret == True:
        print('found!{}: imgpoints id:{}'.format(i, len(objpoints)))
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners)
        if save_cornerimg:
            show = img.copy()
            cv2.drawChessboardCorners(show, pattern_size, corners, ret) 
            fname = join(load_folder, 'corner{}.jpg'.format(len(imgpoints)-1))
            cv2.imwrite(fname, show)
    else:
        print("couldn't find corner")

In [ ]:
print("#### CALIB START #####")
N_OK = len(imgpoints)
print("Found " + str(N_OK) + " valid images for calibration")
K = np.zeros((3, 3))
D = np.zeros((5, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
rms, K, D, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, imgsize[::-1], K, D, rvecs, tvecs)
print('final result')
print("K=np.array(" + str(K.tolist()) + ")")
print("D=np.array(" + str(D.tolist()) + ")")
print("RMS:", rms)
print("#### CALIB END #####")

In [ ]:
Camera.setup: "monocular"
Camera.model: "perspective"

Camera.fx: 458.654
Camera.fy: 457.296
Camera.cx: 367.215
Camera.cy: 248.375

Camera.k1: -0.28340811
Camera.k2: 0.07395907
Camera.p1: 0.00019359
Camera.p2: 1.76187114e-05
Camera.k3: 0.0


In [ ]:
def printForOpenVSLAM(K, D, img, fps=0, load_folder=load_folder):
    data = {}
    data['fx'] = K[0][0]
    data['fy'] = K[1][1]
    data['cx'] = K[0][2]
    data['cy'] = K[1][2]

    data['k1'] = D[0][0]
    data['k2'] = D[1][0]
    data['p1'] = D[2][0]
    data['p2'] = D[3][0]
    data['k3'] = D[4][0]
    
    data['fps'] = fps
    data['cols'] = img.shape[1]
    data['rows'] = img.shape[0]
    
    print_array=['Camera.setup: "monocular"',
                 'Camera.model: "perspective"', '']
    for it in ['fx', 'fy', 'cx', 'cy']:
        print_array.append('Camera.{}: {}'.format(it, data[it]))
    print_array.append('')
    for it in ['k1', 'k2', 'p1', 'p2', 'k3']:
        print_array.append('Camera.{}: {}'.format(it, data[it]))
    print_array.append('')
    for it in ['fps', 'cols', 'rows']:
        print_array.append('Camera.{}: {}'.format(it, data[it]))

    with open(join(load_folder,'calib_pinhole_results.txt'), mode='w') as f:
        for it in print_array:
            print(it)
            print(it, file=f)

In [ ]:
printForOpenVSLAM(K, D, img, fps=30, load_folder=load_folder)

In [ ]:
#writes array to .yml file
yml_file = join(load_folder,'calib_pinhole_results.yml')
fs_write = cv2.FileStorage(yml_file, cv2.FILE_STORAGE_WRITE)
fs_write.write('K', K)
fs_write.write('D', D)
fs_write.write('width', img.shape[1])
fs_write.write('height', img.shape[0])
fs_write.release()

#READ
fs_read = cv2.FileStorage(yml_file, cv2.FILE_STORAGE_READ)
fs_read.getNode('K').mat()    
fs_read.getNode('D').mat()
fs_read.getNode('width').real()    
fs_read.getNode('height').real()